In [1]:
import numpy as np
import torch, torchvision
from torchvision import transforms, datasets, models
from PIL import Image

In [2]:
model = models.resnet18(pretrained=True)
model.fc = torch.nn.Identity()

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


In [3]:
def load_img(filename: str):
  transform = transforms.Compose([
                                  transforms.ToTensor(),
                                  transforms.Resize((224, 224)),
                                  transforms.Normalize((0.5, ), (0.5, ))
  ])
  img = Image.open(filename)
  return transform(img)

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def predict_feature(file: str, model):
  model.eval()
  img = file.unsqueeze(0)
  result = model(img.to(device))
  return result.to('cpu').detach().numpy().copy()

In [5]:
import matplotlib.pyplot as plt
import os
import shutil
from sklearn.cluster import KMeans
import pandas

In [14]:
rootpath = '/content/drive/MyDrive/colab_storage/hololive/holo_total'
files = os.listdir(rootpath)
features_list = []
for file in files:
  imgfile = load_img(os.path.join(rootpath, file))
  print(imgfile.shape)
  print(file)
  features = predict_feature(imgfile, model)
  features_list.append(features[0])
kmeans = KMeans(n_clusters=5, max_iter=500, random_state=0).fit(features_list)
print(kmeans.labels_)

torch.Size([3, 224, 224])
aki.jpg
torch.Size([3, 224, 224])
choco.jpg
torch.Size([3, 224, 224])
ayame.jpg
torch.Size([3, 224, 224])
aqua.jpg
torch.Size([3, 224, 224])
shion.jpg
torch.Size([3, 224, 224])
suisei.jpg
torch.Size([3, 224, 224])
subaru.jpg
torch.Size([3, 224, 224])
sora.jpg
torch.Size([3, 224, 224])
rushia.jpg
torch.Size([3, 224, 224])
pekora.jpg
torch.Size([3, 224, 224])
okayu.jpg
torch.Size([3, 224, 224])
miko.jpg
torch.Size([3, 224, 224])
noel.jpg
torch.Size([3, 224, 224])
mio.jpg
torch.Size([3, 224, 224])
meru.jpg
torch.Size([3, 224, 224])
marine.jpg
torch.Size([3, 224, 224])
korone.jpg
torch.Size([3, 224, 224])
kanata.jpg
torch.Size([3, 224, 224])
haato.jpg
torch.Size([3, 224, 224])
fubuki.jpg
torch.Size([3, 224, 224])
flare.jpg
[0 2 1 0 4 2 4 1 0 2 4 1 3 2 4 2 4 0 4 0 4]


In [15]:
df = pandas.DataFrame({ 'image': files, 'label': kmeans.labels_ })
df.to_csv(os.path.join(rootpath, 'image_labels.csv'))

In [16]:
CLUSTER_DIR = '/content/drive/MyDrive/colab_storage/hololive/clustered'
def classify_imgs():
  df = pandas.read_csv(os.path.join(rootpath, 'image_labels.csv'))
  # if os.path.exists(CLUSTER_DIR):
  #   shutil.rmtree(CLUSTER_DIR)
  
  labels = list(set(df['label'].values))
  print(labels)
  for label in labels:
    newdir = os.path.join(CLUSTER_DIR, str(label))
    if not os.path.exists(newdir):
      os.mkdir(newdir)
    imgs = df[df['label']==label]['image'].values
    for img in imgs:
      origin = os.path.join(rootpath, img)
      copyimg = os.path.join(CLUSTER_DIR, str(label), img)
      shutil.copyfile(origin, copyimg)

In [17]:
classify_imgs()

[0, 1, 2, 3, 4]
